In [1]:
import numpy as np
import pandas as pd

In [2]:
list_tar=[(1,2),(1,3),(1,4),(1,5),(1,6),(2,3),(2,4),(2,5),(2,6),(3,4),(3,5),(3,6),(4,5),(4,6),(5,6)]

In [3]:
from random import choice
from itertools import combinations
def form_tri(lst: list,inp: set):
    line_list=[x for x in lst+[inp] if x != -1]
    combs = list(combinations(line_list, 3))
    for comb in combs:
        (a,b),(c,d),(e,f)=comb
        if len(set([a,b,c,d,e,f]))==3:
            return True
    return False
def create_one_sample():
    output=[]
    for i in range(4):
        pick=choice([x for x in list_tar if x not in output])
        output.append(pick)
    for i in range(5):
        temp_list=[x for x in list_tar if x not in output]
        temp_list1=temp_list.copy()
        for j in range(len(temp_list)):
            if form_tri(output[::2],temp_list[j]):
                temp_list1.remove(temp_list[j])
        if temp_list1==[]:
            pick=choice([x for x in list_tar if x not in output])
        else:
            pick=choice(temp_list1)
        output.append(pick)
        temp_list=[x for x in list_tar if x not in output]
        temp_list1=temp_list.copy()
        for j in range(len(temp_list)):
            if form_tri(output[1::2],temp_list[j]):
                temp_list1.remove(temp_list[j])
        if temp_list1==[]:
            pick=choice([x for x in list_tar if x not in output])
        else:
            pick=choice(temp_list1)
        output.append(pick)
    pick=choice([x for x in list_tar if x not in output])
    output.append(pick)
    return output

In [4]:
games=[create_one_sample()]
game=games[0]
for i in range(int(5e4)-1):
    while game in games:
        game=create_one_sample()
    games.append(game)

In [5]:
from itertools import combinations
def find_pos(inp_list):
    for i in range(2,7):
        lst=inp_list[:i+1]
        combs = list(combinations(lst, 3))
        combs=sorted(combs)
        for comb in combs:
            (a,b),(c,d),(e,f)=comb
            if len(set([a,b,c,d,e,f]))==3:
                return i
    return 7

In [6]:
# 0 stands for the the player one win, 1 stands for the player two win
def determine_winner(inp_list):
    pos_1=find_pos(inp_list[::2])
    pos_2=find_pos(inp_list[1::2])
    if pos_1<=pos_2:
        return 1
    else:
        return 0

In [7]:
train_original=[]
for game in games:
    if determine_winner(game)==1:
        pos=find_pos(game[::2])
        game_parsed=game.copy()
        game_parsed[2*pos+1:]=[-1]*len(game_parsed[2*pos+1:])
        train_original.append(game_parsed)
print(len(train_original))
        

24906


In [8]:
enu=enumerate(list_tar)
encode_dict={tar:idx for idx,tar in enu}
encode_dict[-1]=-1
def encode(inp_list):
    return [encode_dict[tar] for tar in inp_list]
decode_dict={idx:tar for idx,tar in enu}
decode_dict[-1]=-1
def decode(inp_list):
    return [decode_dict[tar] for tar in inp_list]

In [9]:
train_encode=[encode(game) for game in train_original]

In [10]:
sample_game=train_encode[0]
sample_game

[9, 13, 4, 0, 5, 1, 12, 2, 8, 10, 3, 7, 6, -1, -1]

In [11]:
def break_down_train(game):
    output_list_input=[]
    output_list_label=[]
    for i in range(0,len(game)-1,2):
        if game[min(i+2,14)]==-1:
            break  
        output_list_input.append(game[:i+1]+[-1]*(15-i-1))
        output_list_label.append(game[i+1])
    return output_list_input,output_list_label

In [12]:
break_down_train(sample_game)

([[9, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
  [9, 13, 4, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
  [9, 13, 4, 0, 5, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
  [9, 13, 4, 0, 5, 1, 12, -1, -1, -1, -1, -1, -1, -1, -1],
  [9, 13, 4, 0, 5, 1, 12, 2, 8, -1, -1, -1, -1, -1, -1],
  [9, 13, 4, 0, 5, 1, 12, 2, 8, 10, 3, -1, -1, -1, -1]],
 [13, 0, 1, 2, 10, 7])

In [38]:
train_input=[]
train_label=[]
for game in train_encode:
    inp,label=break_down_train(game)
    train_input.extend(inp)
    train_label.extend(label)
import torch
train_label_tensor = torch.tensor(train_label)
train_label_encoded = torch.nn.functional.one_hot(train_label_tensor).tolist()
train_label=train_label_encoded
train_input_new=[]
train_label_new=[]
for i in range(len(train_label)):
    if train_input[i] not in train_input_new:
        train_input_new.append(train_input[i])
        train_label_new.append(train_label[i])
    else:
        idx=train_input_new.index(train_input[i])
        train_label_new[idx]=[train_label_new[idx][j]+train_label[i][j] for j in range(15)]

In [39]:
for i in range(len(train_label_new)):
    sum1=sum(train_label_new[i])
    train_label_new[i]=[train_label_new[i][j]/sum1 for j in range(len(train_label_new[i]))]

In [40]:

data={}
for i in range(15):
    data['label_'+str(i)]=[inp1[i] for inp1 in train_label_new]
for i in range(15):
    data['input_'+str(i)]=[inp[i] for inp in train_input_new]
df=pd.DataFrame(data)
df.to_csv('train_one_hotted.csv')